In [4]:
%load_ext autoreload
%autoreload 2
    
import numpy as np
import os
import seaborn as sns
import sys
from functions import *

colors = ['#b3e2cd', '#fdcdac']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Figure 7g + Supplementary Figure 11d: R^2 for conjunctive regression model, across network models

In [ ]:
with open('/.../processed_data/conj_coding.pickle', 'rb') as handle:
    all_data = pickle.load(handle)
    

data_nonlinearMix = {'subtractive': {}, 'divisive_2': {}}

for dend_nonlinearity in ['subtractive', 'divisive_2']:
    print(dend_nonlinearity)
    rsqr_conjs = []
    rsqr_conjs_nosst = []
    for data in all_data:
        if data['hp']['dend_nonlinearity']==dend_nonlinearity: 
            rsqr_conjs.extend(data['rsqr_conjs'])
            rsqr_conjs_nosst.extend(data['rsqr_conjs_nosst'])
    y = [rsqr_conjs, rsqr_conjs_nosst]
    colors = ['#b3e2cd', '#fdcdac']
    fig, ax = plt.subplots(figsize=[3.5, 5])
    ax.plot([0, 1], y, color='k', alpha=0.05, marker='o')
    make_pretty_axes(ax)
    ax.set_xlim([-0.2, 1.2])
    ax.set_xticks([0, 1])
    fig.tight_layout()
    plt.show()
    # fig.savefig('/home/yl4317/Documents/two_module_rnn/figs/conj_rsquare_{}.pdf'.format(dend_nonlinearity))

    # statistical test
    print(stats.ttest_ind(a=rsqr_conjs, b=rsqr_conjs_nosst, alternative='greater'))
    print('n={}'.format(len(rsqr_conjs)))

    # save source data    
    # data_nonlinearMix[dend_nonlinearity] = {'intact': rsqr_conjs, 'silence_sst': rsqr_conjs_nosst}
    # pd.DataFrame.from_dict(data=data_nonlinearMix[dend_nonlinearity], orient='columns').to_csv('/home/yl4317/Documents/two_module_rnn/source_data/nonlinear_mix_rsquares{}.csv'.format(dend_nonlinearity), header=False)
    

# Supplementary Figure 12: the prediction of the regression model before and after silencing SST units, for some example units 

In [ ]:
with open('/.../conj_coding.pickle', 'wb') as f:
    pickle.dump(all_data, f)

data_suppfig12 = {'intact': {}, 'silenceSST': {}}

neuron_ids = [9, 21, 26, 50, 55]    # pick some example unts

for data in all_data:
    print(data['model_name'])
    if data['model_name'] != 'success_2023-05-10-14-28-42_wcst_116_sparsity0':
        continue    # pick an example network
    r_sqr_conjs = data['rsqr_conjs']    # r-square values for the regression model with conjunctive term
    rsqr_conjs_nosst = data['rsqr_conjs_nosst']    # same but for when SST neurons are silenced
    
    predicted_activities = data['predicted_activities']    # predicted residual activity by the regression model with the conjunctive terms across all neurons. Get rid of the transient activity for the first few trials
    residual_activities = data['residual_activities']   # actual residual activity across all neurons
    predicted_activities_nosst = data['predicted_activities_nosst']    # same but when SST are silenced
    residual_activities_nosst = data['residual_activities_nosst']
    
    # plot the prediction versus actual residual activity
    n_neurons = predicted_activities.shape[0]
    for n in range(n_neurons):
        if n not in neuron_ids:
            continue    
        if r_sqr_conjs[n]== 0 or rsqr_conjs_nosst[n] == 0:
            print('r-square = 0 due to inactivity of this unit')
            continue
        fig, ax = plt.subplots(1, 1, figsize=[10, 3])
        fig.suptitle('Intact network, unit {}\nR-square={}'.format(n, r_sqr_conjs[n]))
        ax.plot(predicted_activities[n], label='predicted residual activity')
        ax.plot(residual_activities[n], label='actual residual activity')
        ax.legend()
        ax.legend()
        make_pretty_axes(ax)
        fig.tight_layout()
        plt.show()
        data_suppfig12['intact'][n] = {}
        data_suppfig12['intact'][n]['predicted'] = predicted_activities[n]
        data_suppfig12['intact'][n]['actual'] = residual_activities[n]

        
        fig, ax = plt.subplots(1, 1, figsize=[10, 3])
        fig.suptitle('SST silenced, unit {}\nR-square={}'.format(n, rsqr_conjs_nosst[n]))
        ax.plot(predicted_activities_nosst[n], label='predicted residual activity')
        ax.plot(residual_activities_nosst[n], label='actual residual activity')
        ax.legend()
        ax.legend()
        make_pretty_axes(ax)
        fig.tight_layout()
        plt.show()
        data_suppfig12['silenceSST'][n] = {}
        data_suppfig12['silenceSST'][n]['predicted'] = predicted_activities[n]
        data_suppfig12['silenceSST'][n]['actual'] = residual_activities[n]

